# Notebook 2: Use SageMaker Training Jobs to Accelerate Model Development

## Learning Objectives
- Use SageMaker processing and training jobs to optimize cost and performance
- Compare model performance using SageMaker MLflow
- Deploy a model as an endpoint for real-time inference

## Environment Notes
This notebook was last tested on a `ml.m5.large (2 vCPU + 8 GiB)` instance running the `Python 3` kernel in SageMaker Studio.

---

## 1. Background
In notebook 1 of this series, we demonstrated using RNAseq data to predict HER2 status using the compute resources on the notebook server. However, using notebook server resources to process large amounts of data or train complex models is generally not a good idea. It's possible to scale up your notebook server, but any time you spend on non-compute intensive tasks (i.e. most of your time) will be wasted. A better idea is to run your notebook on a small server and submit compute-intensive tasks to independent jobs. SageMaker provides managed services for running data processing, model training, and hyperparameter tuning jobs. In this notebook, we'll demonstrate how to leverage these services to optimize the performance and cost of our tasks.

Specifically, we'll demonstrate two best practices: **Jobs** and **Experiments**.

These best practices play a key role in the **Prepare Data** and **Model Development** phases of the Machine Learning Lifecycle. For more information, please refer to the [Machine Learning Best Practices in Healthcare and Life Sciences Whitepaper](https://d1.awsstatic.com/whitepapers/ML-best-practices-health-science.pdf?did=wp_card&trk=wp_card).

![Machine Learning Life Cycle - Part 1](img/MLLC1.png "ML Life Cycle - Part 1")

---

## 1.1. SageMaker Jobs

[SageMaker](https://docs.aws.amazon.com/sagemaker/latest/dg/whatis.html) processing, training, and hyperparameter optimization (HPO) jobs allow data scientists to submit compute-heavy processes to external services. This keeps costs optimized and ensures that these tasks run in reproducible environments. It also improves data scientist productivity by allowing these jobs to run in "the background" and provides resiliancy if something happens to your notebook environment.

![alt text](img/jobs.png "Jobs")

## 1.2. MLflow

![MLflow workflow](img/mlflow-diagram.png "MLflow")

[Amazon SageMaker with MLflow](https://docs.aws.amazon.com/sagemaker/latest/dg/mlflow.html) is a capability of Amazon SageMaker that lets you create, manage, analyze, and compare your machine learning experiments. 

Machine learning is an iterative process that requires experimenting with various combinations of data, algorithms, and parameters, while observing their impact on model accuracy. The iterative nature of ML experimentation results in numerous model training runs and versions, making it challenging to track the best performing models and their configurations. The complexity of managing and comparing iterative training runs increases with generative artificial intelligence (generative AI), where experimentation involves not only fine-tuning models but also exploring creative and diverse outputs. Researchers must adjust hyperparameters, select suitable model architectures, and curate diverse datasets to optimize both the quality and creativity of the generated content. Evaluating generative AI models requires both quantitative and qualitative metrics, adding another layer of complexity to the experimentation process.

Use MLflow with Amazon SageMaker to track, organize, view, analyze, and compare iterative ML experimentation to gain comparative insights and register and deploy your best performing models.

---
## 2. Preparation

Let's start by specifying:

- The Python libraries that we'll use throughout the analysis
- The S3 bucket and prefix that you want to use for training and model data.  This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these.  Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the boto regexp with a the appropriate full IAM role arn string(s).

### 2.1. Import Python Libraries

In [1]:
%pip install --disable-pip-version-check -U 'boto3==1.42.9' 'sagemaker==2.255.0' 'mlflow==3.1.4' 'sagemaker-mlflow==0.2.0'

Note: you may need to restart the kernel to use updated packages.


In [2]:
import boto3
import os
import sagemaker
from sagemaker.processing import FrameworkProcessor, ProcessingOutput
from sagemaker.sklearn.estimator import SKLearn
from sagemaker.xgboost.estimator import XGBoost
from sagemaker.tensorflow import TensorFlow
from time import strftime

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


### 2.2. Create Some Necessary Clients

In [3]:
boto_session = boto3.session.Session()
region = boto_session.region_name
sagemaker_session = sagemaker.session.Session(boto_session)
sagemaker_execution_role = sagemaker.session.get_execution_role(sagemaker_session)
sagemaker_boto_client = boto_session.client("sagemaker")
s3_boto_client = boto_session.client("s3")
account_id = boto_session.client("sts").get_caller_identity().get("Account")
print(f"Assumed SageMaker role is {sagemaker_execution_role}")

Assumed SageMaker role is arn:aws:iam::167428594774:role/HCLS-ID-SageMakerExecutionRole-XiOZEqEi1g2V


### 2.3. Specify S3 Bucket and Prefix

In [4]:
S3_BUCKET = sagemaker_session.default_bucket()
S3_PREFIX = "brca-her2-classifier"
S3_PATH = sagemaker.s3.s3_path_join(S3_BUCKET, S3_PREFIX)
print(f"S3 path is {S3_PATH}")

S3 path is sagemaker-us-east-1-167428594774/brca-her2-classifier


### 2.4. Define Local Working Directories

In [5]:
WORKING_DIR = os.getcwd()
DATA_DIR = os.path.join(WORKING_DIR, "data")
print(f"Working directory is {WORKING_DIR}")
print(f"Data directory is {DATA_DIR}")

Working directory is /home/sagemaker-user/aws-healthcare-lifescience-ai-ml-sample-notebooks/workshops/RNAseq_Tertiary_Analysis
Data directory is /home/sagemaker-user/aws-healthcare-lifescience-ai-ml-sample-notebooks/workshops/RNAseq_Tertiary_Analysis/data


### 2.5. Define MLflow parameters

If you haven't yet, create an [MLflow app](https://docs.aws.amazon.com/sagemaker/latest/dg/mlflow-app-setup.html). Then, run the following cell to verfiy that your MLflow server is ready for use.

In [6]:
running_mlflow_apps = [
    summary
    for summary in sagemaker_boto_client.list_mlflow_apps().get(
        "Summaries"
    )
    if summary.get("Status") == "Created"
]
print(running_mlflow_apps)
mlflow_app_arn = [
    server["Arn"] for server in running_mlflow_apps
][-1]
running_mlflow_apps
mlflow_app_arn

[{'Arn': 'arn:aws:sagemaker:us-east-1:167428594774:mlflow-app/app-KVEL4GSJFAAL', 'Name': 'DefaultMLFlowApp', 'Status': 'Created', 'CreationTime': datetime.datetime(2025, 12, 16, 15, 19, 47, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2025, 12, 16, 15, 21, 54, 360000, tzinfo=tzlocal()), 'MlflowVersion': '3.4.0'}]


'arn:aws:sagemaker:us-east-1:167428594774:mlflow-app/app-KVEL4GSJFAAL'

---
## 3. Data Preparation  with Amazon SageMaker Processing

Amazon SageMaker Processing allows you to run steps for data pre- or post-processing, feature engineering, data validation, or model evaluation workloads on Amazon SageMaker. Processing jobs accept data from Amazon S3 as input and store data into Amazon S3 as output.

![processing](https://sagemaker.readthedocs.io/en/stable/_images/amazon_sagemaker_processing_image1.png)

Here, we'll import the dataset and transform it with SageMaker Processing, which can be used to process terabytes of data in a SageMaker-managed cluster separate from the instance running your notebook server. In a typical SageMaker workflow, notebooks are only used for prototyping and can be run on relatively inexpensive and less powerful instances, while processing, training and model hosting tasks are run on separate, more powerful SageMaker-managed instances.  SageMaker Processing includes off-the-shelf support for Scikit-learn, as well as a Bring Your Own Container option, so it can be used with many different data transformation technologies and tasks.    

To use SageMaker Processing, simply supply a Python data preprocessing script as shown below.  For this example, we're using a SageMaker prebuilt Scikit-learn container, which includes many common functions for processing data.  There are few limitations on what kinds of code and operations you can run, and only a minimal contract:  input and output data must be placed in specified directories.  If this is done, SageMaker Processing automatically loads the input data from S3 and uploads transformed data back to S3 when the job is complete.

For this example, we'll download the raw data directly from Xenahubs as part of the processing script, so we do not need to specify an input bucket.

### 3.1. Submit SageMaker Processing Job

This will take about 5 minutes to complete. Notice that this code block references `scripts/processing/processing.py`. The processing job will run this script on a different compute instance, in this case a ml.m5.xlarge. This allows us to use a small instance for our notebook server, while still taking advantage of a more powerful instance for the processing.

In [7]:
HISEQ_URL = "https://tcga.xenahubs.net/download/TCGA.BRCA.sampleMap/HiSeqV2_PANCAN.gz"
BRCA_CLINICAL_MATRIX_URL = (
    "https://tcga.xenahubs.net/download/TCGA.BRCA.sampleMap/BRCA_clinicalMatrix"
)
processing_run_name = f"data-processing-job-{strftime('%Y-%m-%d-%H-%M-%S')}"
train_test_split_ratio = 0.2
gene_count = 20000

sklearn_processor = FrameworkProcessor(
    estimator_cls=SKLearn,
    framework_version="1.2-1",
    instance_count=1,
    instance_type="ml.m5.xlarge",
    role=sagemaker_execution_role,
    sagemaker_session=sagemaker_session,
)

sklearn_processor.run(
    job_name=processing_run_name,
    code="scripts/processing/processing.py",
    dependencies=["scripts/processing/requirements.txt"],
    outputs=[
        ProcessingOutput(
            output_name="train",
            source="/opt/ml/processing/output/train",
            destination=f"s3://{S3_BUCKET}/{S3_PREFIX}/data/train/",
        ),
        ProcessingOutput(
            output_name="validation",
            source="/opt/ml/processing/output/val",
            destination=f"s3://{S3_BUCKET}/{S3_PREFIX}/data/val/",
        ),
        ProcessingOutput(
            output_name="test",
            source="/opt/ml/processing/output/test",
            destination=f"s3://{S3_BUCKET}/{S3_PREFIX}/data/test/",
        ),
    ],
    arguments=[
        "--brca_clinical_matrix_url",
        BRCA_CLINICAL_MATRIX_URL,
        "--hiseq_url",
        HISEQ_URL,
        "--train_test_split_ratio",
        str(train_test_split_ratio),
        "--gene_count",
        str(gene_count),
        "--create_test_data",
    ],
    wait=True,
)

INFO:sagemaker:Creating processing-job with name data-processing-job-2025-12-16-17-25-23


.............CodeArtifact repository not specified. Skipping login.
Found existing installation: typing 3.7.4.3
Uninstalling typing-3.7.4.3:
  Successfully uninstalled typing-3.7.4.3
INFO: pip is looking at multiple versions of sagemaker-core to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of sagemaker-core to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 72.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 219.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.8/750.8 kB 53.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### 3.2. Download Processed Data from S3

In [8]:
sagemaker_session.download_data(
    f"{DATA_DIR}/output/train",
    bucket=S3_BUCKET,
    key_prefix=f"{S3_PREFIX}/data/train/train.csv",
)
sagemaker_session.download_data(
    f"{DATA_DIR}/output/val",
    bucket=S3_BUCKET,
    key_prefix=f"{S3_PREFIX}/data/val/val.csv",
)
sagemaker_session.download_data(
    f"{DATA_DIR}/output/test",
    bucket=S3_BUCKET,
    key_prefix=f"{S3_PREFIX}/data/test/test.csv",
)

['/home/sagemaker-user/aws-healthcare-lifescience-ai-ml-sample-notebooks/workshops/RNAseq_Tertiary_Analysis/data/output/test/test.csv']

---
## 4. Model Training

Now that our training data is set up, we can train some models. To highlight the benefits of experiment tracking, we're going to train models using three different frameworks:
- The random forest model from Scikit Learm
- A multi-layer perceptron (MLP) neural network in Keras
- The open-source XGBoost algorithm

Since we're using SageMaker jobs to run our training, we don't need to install any additional libraries or spin up expensive compute resources on our notebook server. The jobs use their own dependencies and we're only charged for the time they run.

First, let's define some variables that all three training jobs will need.

In [9]:
# define the data type and paths to the training and validation datasets
content_type = "text/csv"

s3_input_train = sagemaker.inputs.TrainingInput(
    f"s3://{S3_BUCKET}/{S3_PREFIX}/data/train/train.csv", content_type=content_type
)

s3_input_validation = sagemaker.inputs.TrainingInput(
    f"s3://{S3_BUCKET}/{S3_PREFIX}/data/val/val.csv", content_type=content_type
)

model_output_path = f"s3://{S3_BUCKET}/{S3_PREFIX}/models/"

### 4.1. Train Model Using a SKLearn Random Forest Algorithm

Here again we're passing a script (`scripts/rf_train/rf_train.py`) to run during the training job. Notice that we've also included a `requirements.txt` file in the training script directory to install additional dependencies in the training container. This is a great way to install an extra package or two without creating your own container image from scratch!

Setting `wait=False` allows us to continue running the notebook while the training job runs in "the background" (on a different machine).

In [10]:
rf_job_name = f"RF-Training-Job-{strftime('%Y-%m-%d-%H-%M-%S')}"

n_estimators = 100
min_samples_leaf = 3

rf_estimator = SKLearn(
    base_job_name=rf_job_name,
    enable_sagemaker_metrics=True,
    entry_point="rf_train.py",
    framework_version="1.2-1",
    hyperparameters={
        "n-estimators": n_estimators,
        "min-samples-leaf": min_samples_leaf,
    },
    instance_count=1,
    instance_type="ml.m5.xlarge",
    output_path=model_output_path,
    role=sagemaker_execution_role,
    sagemaker_session=sagemaker_session,
    source_dir="scripts/rf_train",
    environment={"MLFLOW_TRACKING_URI": mlflow_app_arn},
)
rf_estimator.fit(
    {"train": s3_input_train, "validation": s3_input_validation},
    job_name=rf_job_name,
    wait=False,
)

INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker:Creating training-job with name: RF-Training-Job-2025-12-16-17-30-50


---

### 4.2. Train Model using a Keras MLP

In [11]:
tf_job_name = f"TF-Training-Job-{strftime('%Y-%m-%d-%H-%M-%S')}"

tf_estimator = TensorFlow(
    enable_sagemaker_metrics=True,
    entry_point="tf_train.py",
    framework_version="2.16",
    instance_count=1,
    instance_type="ml.m5.xlarge",
    metric_definitions=[
        {"Name": "validation:accuracy", "Regex": "Validation Accuracy: ([0-9.]+)$"},
        {"Name": "validation:precision", "Regex": "Validation Precision: ([0-9.]+)$"},
        {"Name": "validation:f1", "Regex": "Validation F1 Score: ([0-9.]+)$"},
    ],
    output_path=model_output_path,
    py_version="py310",
    role=sagemaker_execution_role,
    sagemaker_session=sagemaker_session,
    source_dir="scripts/tf_train",
    environment={"MLFLOW_TRACKING_URI": mlflow_app_arn},
)

tf_estimator.fit(
    {"train": s3_input_train, "validation": s3_input_validation},
    job_name=tf_job_name,
    wait=False,
)

INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: TF-Training-Job-2025-12-16-17-30-51


### 4.3. Train Model Using the XGBoost Algorithm

Compare the XGBoost training script we're about to run (scripts/rf_train/rf_train.py) with the training function we used in Notebook 1. You'll notice that the `xgb.train` call is the same in both!

Since we're setting `wait=True` our Jupyter session will wait until this training job is finished before moving on

Submit the training job

In [12]:
xgb_job_name = f"XGB-Training-Job-{strftime('%Y-%m-%d-%H-%M-%S')}"

framework_version = "1.7-1"
py_version = "py3"

hyper_params_dict = {
    "objective": "binary:logistic",
    "booster": "gbtree",
    "eval_metric": "error",
    "scale_pos_weight": 9.0,
    "max_depth": 3,
    "min_child_weight": 5,
    "subsample": 0.9,
    "verbosity": 1,
    "tree_method": "auto",
}

xgb_estimator = XGBoost(
    enable_sagemaker_metrics=True,
    entry_point="xgb_train.py",
    framework_version=framework_version,
    hyperparameters=hyper_params_dict,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    output_path=model_output_path,
    py_version=py_version,
    role=sagemaker_execution_role,
    sagemaker_session=sagemaker_session,
    source_dir="scripts/xgb_train",
    environment={"MLFLOW_TRACKING_URI": mlflow_app_arn}
)

xgb_estimator.fit(
    {"train": s3_input_train, "validation": s3_input_validation},
    job_name=xgb_job_name,
    logs=True,
    wait=True,
)

INFO:sagemaker.image_uris:Ignoring unnecessary Python version: py3.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: ml.m5.xlarge.
INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker:Creating training-job with name: XGB-Training-Job-2025-12-16-17-30-53


2025-12-16 17:30:54 Starting - Starting the training job...
2025-12-16 17:31:27 Starting - Preparing the instances for training...
2025-12-16 17:31:50 Downloading - Downloading input data...
2025-12-16 17:32:20 Downloading - Downloading the training image......
2025-12-16 17:33:06 Training - Training image download completed. Training in progress./miniconda3/lib/python3.9/site-packages/sagemaker_containers/_server.py:22: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
[2025-12-16 17:33:17.131 ip-10-0-150-100.ec2.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2025-12-16 17:33:17.189 ip-10-0-150-100.ec2.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2025-12-16:17:33:17:INFO] Imported framework sagemaker_xgboost_container.traini

---
## 5. Model Evaluation

### 5.1. Compare Model Results in MLflow

In [13]:
import mlflow

mlflow.set_tracking_uri(mlflow_app_arn)
runs = mlflow.search_runs(search_all_experiments=True)

display(runs)

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.training_score,metrics.training_recall_score,metrics.training_accuracy_score,metrics.training_roc_auc,...,params.opt_clipnorm,params.opt_beta_2,tags.mlflow.source.name,tags.mlflow.user,tags.mlflow.runName,tags.estimator_name,tags.mlflow.source.type,tags.estimator_class,tags.mlflow.autologging,tags.mlflow.latest_checkpoint_artifact
0,0ca6c7b21f54451aba56d689f52f6d68,0,FINISHED,s3://sagemaker-us-east-1-167428594774/0/0ca6c7...,2025-12-16 17:34:11.837000+00:00,2025-12-16 17:34:17.765000+00:00,1.0,1.0,1.0,1.0,...,None,None,rf_train.py,root,nebulous-mule-963,RandomForestClassifier,LOCAL,sklearn.ensemble._forest.RandomForestClassifier,sklearn,None
1,7301b7b4144d44fead22b5bcfba16d19,0,FINISHED,s3://sagemaker-us-east-1-167428594774/0/7301b7...,2025-12-16 17:33:45.351000+00:00,2025-12-16 17:34:10.092000+00:00,NaN,NaN,NaN,NaN,...,None,None,/opt/ml/code/xgb_train.py,root,wistful-ape-656,None,LOCAL,None,xgboost,None
2,43f23c2924924c45b25bf8e2e1b24b6d,0,FINISHED,s3://sagemaker-us-east-1-167428594774/0/43f23c...,2025-12-16 17:33:36.032000+00:00,2025-12-16 17:34:28.275000+00:00,NaN,NaN,NaN,NaN,...,None,0.999,tf_train.py,root,dazzling-rat-890,None,LOCAL,None,tensorflow,checkpoints/latest_checkpoint.h5
3,cb9d7b71382b4f338228ea453175c1e8,0,FINISHED,s3://sagemaker-us-east-1-167428594774/0/cb9d7b...,2025-12-16 16:50:05.280000+00:00,2025-12-16 16:50:18.695000+00:00,NaN,NaN,NaN,NaN,...,None,None,/opt/ml/code/xgb_train.py,root,rumbling-croc-155,None,LOCAL,None,xgboost,None
4,787c4fc3bcc44b6abf79907b90caefb5,0,FINISHED,s3://sagemaker-us-east-1-167428594774/0/787c4f...,2025-12-16 16:49:45.814000+00:00,2025-12-16 16:50:21.213000+00:00,NaN,NaN,NaN,NaN,...,None,0.999,tf_train.py,root,flawless-mare-868,None,LOCAL,None,tensorflow,checkpoints/latest_checkpoint.h5
5,f7132d5943424d7ba67f525b69967566,0,FINISHED,s3://sagemaker-us-east-1-167428594774/0/f7132d...,2025-12-16 16:49:40.757000+00:00,2025-12-16 16:49:56.189000+00:00,1.0,1.0,1.0,1.0,...,None,None,rf_train.py,root,enchanting-rat-523,RandomForestClassifier,LOCAL,sklearn.ensemble._forest.RandomForestClassifier,sklearn,None
6,a658306ad0924c72b90ee0ba1c2ab82d,0,FINISHED,s3://sagemaker-us-east-1-167428594774/0/a65830...,2025-12-16 15:57:36.142000+00:00,2025-12-16 15:57:50.380000+00:00,NaN,NaN,NaN,NaN,...,None,None,/opt/ml/code/xgb_train.py,root,capable-elk-430,None,LOCAL,None,xgboost,None
7,e3ea158ef94e42edb94c6c3d8fe22a39,0,FINISHED,s3://sagemaker-us-east-1-167428594774/0/e3ea15...,2025-12-16 15:40:54.099000+00:00,2025-12-16 15:41:08.792000+00:00,NaN,NaN,NaN,NaN,...,None,None,/opt/ml/code/xgb_train.py,root,secretive-goose-548,None,LOCAL,None,xgboost,None


Finally, we'll look at options for deploying this model.

---
## 6. Deploy

Now that we've trained a model, we can allow other applications to use it for inference by deploying it. SageMaker offers several deployment options, based on your performance, cost, and data needs:

![alt text](img/deployment_options.png "SageMaker Model Deployment Options")

### 6.1. Deploy Model as SageMaker Endpoint

Real-time inference endpoints are deployed to a persistent EC2 instance. This allows them to respond quickly to requests and support a wide range of custom properties. It's a good choice for models with steady usage. Deploying this endpoint will take about 5 minutes.

In [14]:
from sagemaker.xgboost import XGBoostModel
from time import strftime
import sagemaker

# Create a model object from the training job
xgb_model = XGBoostModel(
    model_data=xgb_estimator.model_data,
    role=sagemaker_execution_role,
    entry_point="inference.py",  # Use the lightweight inference script
    source_dir="scripts/xgb_train",
    framework_version="1.7-1",
    py_version="py3",
    sagemaker_session=sagemaker_session,
)

# Deploy the model
realtime_endpoint_name = f"her2-real-time-endpoint-{strftime('%Y-%m-%d-%H-%M-%S')}"
xgb_predictor = xgb_model.deploy(
    endpoint_name=realtime_endpoint_name,
    serializer=sagemaker.serializers.CSVSerializer(),
    deserializer=sagemaker.deserializers.JSONDeserializer(),
    instance_type="ml.m5.large",
    initial_instance_count=1,
    wait=True,
)

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: ml.m5.large.
INFO:sagemaker:Creating model with name: sagemaker-xgboost-2025-12-16-17-34-42-611
INFO:sagemaker:Creating endpoint-config with name her2-real-time-endpoint-2025-12-16-17-34-42
INFO:sagemaker:Creating endpoint with name her2-real-time-endpoint-2025-12-16-17-34-42


-------!

### 6.2. Test Endpoint

In [15]:
import pandas as pd
from time import sleep

# Load a random sample of 10 records from the test data
test_df = pd.read_csv("data/output/test/test.csv", header=None).sample(n=25)

# Submit the 10 samples to the inference endpoint and compare the actual and predicted values
print(
    f"Sending test traffic to the endpoint {xgb_predictor.endpoint_name}. \nPlease wait..."
)

for i, row in test_df.iterrows():
    print(
        f"[Actual | predicted] labels for record {i:3} are [{row[0]} | {xgb_predictor.predict([row.iloc[1:]])[0]:.3f}]"
    )
    sleep(0.1)

Sending test traffic to the endpoint her2-real-time-endpoint-2025-12-16-17-34-42. 
Please wait...
[Actual | predicted] labels for record   1 are [0.0 | 0.003]
[Actual | predicted] labels for record 106 are [0.0 | 0.002]
[Actual | predicted] labels for record  80 are [0.0 | 0.001]
[Actual | predicted] labels for record  35 are [0.0 | 0.004]
[Actual | predicted] labels for record  74 are [0.0 | 0.202]
[Actual | predicted] labels for record  25 are [0.0 | 0.063]
[Actual | predicted] labels for record  13 are [0.0 | 0.005]
[Actual | predicted] labels for record  91 are [1.0 | 0.977]
[Actual | predicted] labels for record  94 are [0.0 | 0.002]
[Actual | predicted] labels for record  97 are [1.0 | 0.736]
[Actual | predicted] labels for record  34 are [0.0 | 0.020]
[Actual | predicted] labels for record  48 are [1.0 | 0.951]
[Actual | predicted] labels for record  62 are [0.0 | 0.020]
[Actual | predicted] labels for record  19 are [0.0 | 0.009]
[Actual | predicted] labels for record  52 are [

### 6.3. Clean Up

In [16]:
# Delete endpoint
xgb_predictor.delete_endpoint()

# Delete all S3 objects
bucket = boto_session.resource("s3").Bucket(S3_BUCKET)
bucket.objects.filter().delete()

import shutil

shutil.rmtree("data", ignore_errors=True)
shutil.rmtree("models", ignore_errors=True)
shutil.rmtree("generated", ignore_errors=True)
shutil.rmtree("training_reports", ignore_errors=True)

INFO:sagemaker:Deleting endpoint configuration with name: her2-real-time-endpoint-2025-12-16-17-34-42
INFO:sagemaker:Deleting endpoint with name: her2-real-time-endpoint-2025-12-16-17-34-42
